# About

This code is complementary material to the paper **Optimizing Patient Transitions to Skilled Nursing Facilities** by Alvarez Avendaño et al. Here we implement functions to calculate and compare costs of the myopic policy and the heuristic defined in that paper relative to the optimal policy.





**Julia version: 1.10.0** 

**Required packages:**

```julia
import Pkg;

Pkg.add("PyCall")
Pkg.add("DataFrames")
Pkg.add("XLSX")
Pkg.add("HDF5")
Pkg.add("JLD")
Pkg.add("JSON")

Pkg.add("LaTeXStrings")
```

In [1]:
import Pkg;

Pkg.add("PyCall")
Pkg.add("DataFrames")
Pkg.add("XLSX")
Pkg.add("HDF5")
Pkg.add("JLD")
Pkg.add("JSON")

Pkg.add("LaTeXStrings")

    Updating registry at `C:\Users\Nagato\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\Nagato\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Nagato\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Nagato\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Nagato\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Nagato\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Nagato\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Nagato\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Nagato\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Nagato\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Nagato\.julia\environments\v1.10\Manifest.toml`
   Resolving package versi

# Utilities

Here, we define some utility functions for printing transition probability matrices nicely later in the code.

In [2]:
function pMat2dict(pMat, l)
    digits = 2;
    p_matrices = Dict();
    for j in 0:l
        p_matrices[j] = Dict();
        for a in 0:l
            P = [[round(pMat(a, j, c, d), digits=digits) for d in [0, 1]] for c in [0, 1]];
            p_matrices[j][a] = P;
        end
    end
    return p_matrices;
end

function matrix2latex(matrix, format="matrix"; align=nothing)
    if format == "matrix"
        st = "\\begin{bmatrix}\n";
        nd = "\\end{bmatrix}";
    elseif format == "empty"
        st = "";
        nd = "";
    elseif format == "tabular"
        align_str = join(align, " ");
        st = "\\begin{tabular}{$align_str}\n";
        nd = "\\end{tabular}";
    else
        throw("unknown format '$format'");
    end
    matrix_tex = st;
    matrix_tex *= join([join([string(x) for x in row], "&") for row in matrix], ("\\\\"*"\n"));
    matrix_tex *= "\n"*nd;
    return matrix_tex;
end

function transMat(pm, l)
    function f(a, m, j) 
        return "\\mathbf{P}^{"*string(a) * "," * string(j) * "} &= "*matrix2latex(m)*",";
    end
    tex_arr = [[f(i, pm[j][i], j) for i in 0:l] for j in 0:l];
    ktex = matrix2latex(tex_arr, "empty");
    ktex = "\\begin{align*}\n" * ktex * "\\end{align*}";
    return ktex;
end

function pMat2Latex(pMat, l)
    return transMat(pMat2dict(pMat, l), l);
end

pMat2Latex (generic function with 1 method)

## Construct policies

The state space is given by the set of $(l+1)$-tuples 
$$\mathbb{X}:=\{(i,s_0,s_1,\ldots,s_l)| i \in\{0,1,\ldots,k\}, s_j \in \{0,1\}, j=0,1,\ldots,l\}.$$
where 

* $i$ denotes patient type,
* $k$ is the number of types of patients,
* $s_j$ denotes the state of facility $j$, 
* $l$ is the number of facilities.

In [3]:
function lenStateSpace(k::Int, l::Int)
    #=
    Returns the size of the state space of the system
    k: denotes the number of patients
    l: denotes the number of facilities
    =#
    lenS = (k+1)*(2^(l+1));
    return lenS;
end

function genStateSpace(k::Int, l::Int)
    #=
    Generates the state space 𝕏 of the system
    k: denotes the number of patients
    l: denotes the number of facilities
    =#
    return vec([[a...] for a in collect(Iterators.product(0:k, [0:1 for j in 0:l]...))]);
end

genStateSpace (generic function with 1 method)

In [4]:
function genR(readmissions::Array{Array{Float64}})
    function r(x::Array{Int}, j::Int)
        #=
        This function takes a vector state x=[i, s_0, s_1, ..., s_l],
        where i is the type of patient, and s_k is the state of facility k for all k,
        and a number 0<=j<=l.
        
        This function returns the cost of readmission for patient i in snf j.
        =#
        # first number in the array is type of patient
        i = x[1];
        if i == 0
            # patient type 0 represents no patient at this time, thus, cost is always 0
            return 0;
        end
        if j == 0
            # facility type 0 represents the event "send patient home", and since this option
            # is undesirable, the readmission cost is set very high
            return 100;
        end
        return readmissions[j][i];
    end
    return r;
end

genR (generic function with 1 method)

In [5]:
function genP(λ::Function, pMat::Function, l::Int)
    function p(x1::Array{Int}, x0::Array{Int}, a::Int)
        #=
        This function returns the probability P_{X0, X1}^{a} defined
        as the probability of transitioning from one state x0 to a state
        x1 for SNF a
        =#
        return λ(x1[1])*prod([pMat(a, j, x0[j+2], x1[j+2]) for j in 0:l]);
    end
    return p;
end

genP (generic function with 1 method)

Next step is to construct the transition probability matrices according to the rules for each scenario. The function "constructMatrices" below returns an array of transition probability matrices, which will serve as base matrices to generate transition probability matrices for each scenario. 

\begin{align*}
\begin{bmatrix}
p^{1, 1}_{1, 1}&1-p^{1, 1}_{1, 2}\\
p^{1, 1}_{2, 1}&1-p^{1, 1}_{2, 2}
\end{bmatrix}& &\dots&&\begin{bmatrix}
p^{l, 1}_{1, 1}&1-p^{l, 1}_{1, 2}\\
p^{l, 1}_{2, 1}&1-p^{l, 1}_{2, 2}
\end{bmatrix}\\
\vdots&&\ddots&&\vdots\\
\begin{bmatrix}
p^{1, l}_{1, 1}&1-p^{1, l}_{1, 2}\\
p^{1, 1}_{2, 1}&1-p^{1, l}_{2, 2}
\end{bmatrix}&&\dots&&\begin{bmatrix}
p^{l, l}_{1, 1}&1-p^{l, l}_{1, 2}\\
p^{l, l}_{2, 1}&1-p^{l, l}_{2, 2}
\end{bmatrix}
\end{align*}

In [6]:
function constructMatrices(vec)
    model_p_ce = [
        [[vec[i], 1-vec[i]], [vec[i+1], 1-vec[i+1]]] for i in 1:2:length(vec)
    ]
    return model_p_ce
end

constructMatrices (generic function with 1 method)

the function "getPmatSc1" below returns a function that calculates the transition probability matrices according to the rules described for scenario 1

\begin{align*}
\mathbf{P}^{0,0} &= \begin{bmatrix}
0.0&1.0\\
0.0&1.0
\end{bmatrix},&\mathbf{P}^{1,0} &= \begin{bmatrix}
0.0&1.0\\
0.0&1.0
\end{bmatrix},&&\dots&\mathbf{P}^{3,0} &= \begin{bmatrix}
0.0&1.0\\
0.0&1.0
\end{bmatrix}\\
\mathbf{P}^{0,1} &= \begin{bmatrix}
0.0&1.0\\
0.0&1.0
\end{bmatrix},&\mathbf{P}^{1,1} &= \begin{bmatrix}
p^{1, 1}_{1, 1}&1-p^{1, 1}_{1, 1}\\
p^{1, 1}_{2, 1}&1-p^{1, 1}_{2, 1}
\end{bmatrix},& &\dots&\mathbf{P}^{l,1} &= \begin{bmatrix}
p^{l, 1}_{1, 1}&1-p^{l, 1}_{1, 1}\\
p^{l, 1}_{2, 1}&1-p^{l, 1}_{2, 1}
\end{bmatrix}\\
&\vdots&&\vdots&&\ddots&&\vdots\\
\mathbf{P}^{0,l} &= \begin{bmatrix}
0.0&1.0\\
0.0&1.0
\end{bmatrix},&\mathbf{P}^{1,l} &= \begin{bmatrix}
p^{1, l}_{1, 1}&1-p^{1, l}_{1, 1}\\
p^{1, 1}_{2, 1}&1-p^{1, l}_{2, 1}
\end{bmatrix},&&\dots&\mathbf{P}^{l,l} &= \begin{bmatrix}
p^{l, l}_{1, 1}&1-p^{l, l}_{1, 1}\\
p^{l, l}_{2, 1}&1-p^{l, l}_{2, 1}
\end{bmatrix}
\end{align*}

In [7]:
function getPmatSc1(base, beta_11)
    function pMat(a, j, c, d)
        if a == 0 || j == 0
            p = 1
        else
            mat = base[j]
            if c == 0
                p = mat[1][2]
            elseif c == 1
                p = mat[2][2]*(a!=j) + beta_11*mat[2][2]*(a==j)
            else
                throw("not implemented")
            end
        end
        return float(p*d + (1-p)*(1-d));
    end
    return pMat
end

getPmatSc1 (generic function with 1 method)

In [8]:
function genA(l::Int, genS)
    function A(x::Array{Int})
        #=
        Let x=[i, s1, s2, ...sl] be the state of the system,
        where sk is the state of the SNF k (available:1, unavailable:0).
        This function returns the set of available SNFs, 
        defined as the set {k: sk=1}.
        If there's no available SNF, then returns 0,
        which means that patient is sent home
        =#
        if x[1] == 0
            return [0];
        end
        mA = 2:l+1;
        ls = [y-1 for y in mA if x[y+1] == 1];
        if isempty(ls)
            ls = [0];
        end
        return ls;
    end
    Aev = Dict(x => A(x) for x in genS);
    return function (x) return Aev[x] end;
end

genA (generic function with 1 method)

In [9]:
function genλ(lst)
    #=
    Generates a function that returns arrival rates for each type of patient
    =#
    _lambda_ce_sub = lst
    _lambda_ce_sub = _lambda_ce_sub/sum(_lambda_ce_sub)
    lambda_ce_sub = function (i) return _lambda_ce_sub[i+1] end
    return lambda_ce_sub
end

genλ (generic function with 1 method)

## Policies

In [10]:
function genMyopicPolicy(A::Function, r::Function, λ::Function, genS)
    #=
    This function generates the myopic policy given the cost function (r)
    and the action space A
    =#
    function f(x)
        s = [r(x, a) for a in A(x)];
        return A(x)[argmin(s)];
    end
    Policy = Dict(x => f(x) for x in genS);
    return Policy;
end

genMyopicPolicy (generic function with 1 method)

In [11]:
function genHeuristic2(r::Function, A::Function, genS)
    #=
    This function generates the heuristic 2 policy
    =#
    rCache = Dict(xp => min([r(xp, ap) for ap in A(xp)]...) for xp in genS);
    function heur2(λ::Function, p::Function)
        function compare(x, xp, a)
            rMin = rCache[xp];
            return p(xp, x, a)*rMin;
        end
        function order(x, a)
            res = sum([compare(x, xp, a) for xp in genS]);
            return r(x, a) + res;
        end
        return Dict(x => A(x)[argmin([order(x, a) for a in A(x)])] for x in genS);
    end
    return heur2;
end

genHeuristic2 (generic function with 1 method)

# Policy evaluation

In [12]:
using LinearAlgebra


function genTransitionMatrix(d, genS, p)
    #=
    Generates transition matrices given a function that calculates probabilities
    =#
    P_d = [p(xj, xi, d[xi]) for xi in genS, xj in genS];
    return P_d;
end

function evaluatePolicyARRandomPolicy(Pπ::Array{<:Real, 2}, πr::Vector{<:Real}; ind=Nothing)
    #=
    This function evaluates the expected cost of implementing arbitrary policies "π" given the
    matrix "Pπ", and the cost "rπ"
    =#
    n = size(Pπ, 1);
    if ind == Nothing
        ind = rand(1:n);
    end

    M = (1.0I - Pπ);
    M[:, ind] .= 1;
    invM = inv(M);
    z = invM*πr;
    g = z[ind];
    z[ind] = 0;
    h = z;
    return h, g;
end

function evalPolicyAr(d, genS, r, p, ind=Nothing)
    #=
    This function evaluates the expected cost of implementing arbitrary policies "d" given the state space genS,
    the readmission cost "r", and the function "p" that calculates transition probabilities
    =#
    P_d = genTransitionMatrix(d, genS, p);
    r_d = [r(x, d[x]) for x in genS];
    
    return evaluatePolicyARRandomPolicy(P_d, r_d);
end

evalPolicyAr (generic function with 2 methods)

In [13]:
function genPπ(A::Function, π::Function, genS, r::Function, p::Function)
    #=
    Generates the matrix resulting after multiplying the transition probability matrices P
    with π
    =#
    lenS = length(genS);
    
    Pπ = zeros(lenS, lenS);
    @sync begin
        for (i, xi) in enumerate(genS), (j, xj) in enumerate(genS)
#             Threads.@spawn begin
            begin
                Pπ[i, j] = sum([p(xj, xi, a)*π(a, xi) for a in A(xi)]);
            end
        end
    end
    
    πr = zeros(lenS);
    @sync begin
        for (i, xi) in enumerate(genS)
#             Threads.@spawn begin
            begin
                πr[i] = sum([π(a, xi)*r(xi, a) for a in A(xi)]);
            end
        end
    end
    
    return Pπ, πr;
end

genPπ (generic function with 1 method)

# Policy iteration

In [14]:
function hImprovingAr(h_n, g_n, lenS, genS, A, r, p)
    function tradeOffAr(x0, a)
        return r(x0, a) - g_n + sum(p(genS[i], x0, a)*h_n[i] for i in 1:lenS);
    end
    
    d_np1 = Dict();
    for s in genS
        As = A(s);
        a_opt = As[argmin([tradeOffAr(s, a) for a in As])];
        d_np1[s] = a_opt;
    end
    return d_np1;
end

hImprovingAr (generic function with 1 method)

In [15]:
function policyIterationAr(d_0, lenS, genS, A, r, p; n_max=100, verbose=false)
    #=
    This function implements the policy iteration algorithm, and returns the optimal policy
    given a starting policy "d_0", the state space "genS", the length of the state space "lenS"
    the action space "A", cost "r", and transition probabilities "p"
    =#
    h_n = Nothing;
    g_n = Nothing;
    d_n = d_0;
    
    for it in 1:n_max
        h_n, g_n = evalPolicyAr(d_n, genS, r, p);
        d_np1 = hImprovingAr(h_n, g_n, lenS, genS, A, r, p);
        if d_n == d_np1
            if verbose
                println("\nConverged (iteration={})".format(it));
            end
            break;
        end
        d_n = d_np1;
    end
    return d_n, h_n, g_n;
end

policyIterationAr (generic function with 1 method)

# Run

## Data

In [16]:
# Matrix with readmission rates defined in the paper
readmissions = Array{Array{Float64}}([
    [14.3,  9.5, 19.1, 19.2],
    [16.4, 12.8, 20.1, 20.4],
    [15.6, 12.4, 20.6, 20.2],
    [9.1,   8.7, 11.2, 19.6],
    [20.6,  5.8, 19.0, 13.4],
]);

K = 4;  # Number of types of patients
L = 5;  # Number of SNFs

lenS = lenStateSpace(K, L);  # length of state space
genS = genStateSpace(K, L);  # State space

A = genA(L, genS);  # Action space

r = genR(readmissions);  # Function that returns costs of assigning patient type i to SNF j
λ = genλ([0.2, 0.2, 0.2, 0.2, 0.2]);  # Function that returns arrival rates

mat = constructMatrices([0.1 0.1 0.9 0.1 0.9 0.1 0.9 0.1 0.9 0.1 ]);
pMatSc1 = getPmatSc1(mat, 0.5);
p = genP(λ, pMatSc1, L);  # Transition probability matrices

In [17]:
display("text/markdown", pMat2Latex(pMatSc1, 5))

\begin{align*}
\mathbf{P}^{0,0} &= \begin{bmatrix}
0.0&1.0\\
0.0&1.0
\end{bmatrix},&\mathbf{P}^{1,0} &= \begin{bmatrix}
0.0&1.0\\
0.0&1.0
\end{bmatrix},&\mathbf{P}^{2,0} &= \begin{bmatrix}
0.0&1.0\\
0.0&1.0
\end{bmatrix},&\mathbf{P}^{3,0} &= \begin{bmatrix}
0.0&1.0\\
0.0&1.0
\end{bmatrix},&\mathbf{P}^{4,0} &= \begin{bmatrix}
0.0&1.0\\
0.0&1.0
\end{bmatrix},&\mathbf{P}^{5,0} &= \begin{bmatrix}
0.0&1.0\\
0.0&1.0
\end{bmatrix},\\
\mathbf{P}^{0,1} &= \begin{bmatrix}
0.0&1.0\\
0.0&1.0
\end{bmatrix},&\mathbf{P}^{1,1} &= \begin{bmatrix}
0.1&0.9\\
0.55&0.45
\end{bmatrix},&\mathbf{P}^{2,1} &= \begin{bmatrix}
0.1&0.9\\
0.1&0.9
\end{bmatrix},&\mathbf{P}^{3,1} &= \begin{bmatrix}
0.1&0.9\\
0.1&0.9
\end{bmatrix},&\mathbf{P}^{4,1} &= \begin{bmatrix}
0.1&0.9\\
0.1&0.9
\end{bmatrix},&\mathbf{P}^{5,1} &= \begin{bmatrix}
0.1&0.9\\
0.1&0.9
\end{bmatrix},\\
\mathbf{P}^{0,2} &= \begin{bmatrix}
0.0&1.0\\
0.0&1.0
\end{bmatrix},&\mathbf{P}^{1,2} &= \begin{bmatrix}
0.9&0.1\\
0.1&0.9
\end{bmatrix},&\mathbf{P}^{2,2} &= \begin{bmatrix}
0.9&0.1\\
0.55&0.45
\end{bmatrix},&\mathbf{P}^{3,2} &= \begin{bmatrix}
0.9&0.1\\
0.1&0.9
\end{bmatrix},&\mathbf{P}^{4,2} &= \begin{bmatrix}
0.9&0.1\\
0.1&0.9
\end{bmatrix},&\mathbf{P}^{5,2} &= \begin{bmatrix}
0.9&0.1\\
0.1&0.9
\end{bmatrix},\\
\mathbf{P}^{0,3} &= \begin{bmatrix}
0.0&1.0\\
0.0&1.0
\end{bmatrix},&\mathbf{P}^{1,3} &= \begin{bmatrix}
0.9&0.1\\
0.1&0.9
\end{bmatrix},&\mathbf{P}^{2,3} &= \begin{bmatrix}
0.9&0.1\\
0.1&0.9
\end{bmatrix},&\mathbf{P}^{3,3} &= \begin{bmatrix}
0.9&0.1\\
0.55&0.45
\end{bmatrix},&\mathbf{P}^{4,3} &= \begin{bmatrix}
0.9&0.1\\
0.1&0.9
\end{bmatrix},&\mathbf{P}^{5,3} &= \begin{bmatrix}
0.9&0.1\\
0.1&0.9
\end{bmatrix},\\
\mathbf{P}^{0,4} &= \begin{bmatrix}
0.0&1.0\\
0.0&1.0
\end{bmatrix},&\mathbf{P}^{1,4} &= \begin{bmatrix}
0.9&0.1\\
0.1&0.9
\end{bmatrix},&\mathbf{P}^{2,4} &= \begin{bmatrix}
0.9&0.1\\
0.1&0.9
\end{bmatrix},&\mathbf{P}^{3,4} &= \begin{bmatrix}
0.9&0.1\\
0.1&0.9
\end{bmatrix},&\mathbf{P}^{4,4} &= \begin{bmatrix}
0.9&0.1\\
0.55&0.45
\end{bmatrix},&\mathbf{P}^{5,4} &= \begin{bmatrix}
0.9&0.1\\
0.1&0.9
\end{bmatrix},\\
\mathbf{P}^{0,5} &= \begin{bmatrix}
0.0&1.0\\
0.0&1.0
\end{bmatrix},&\mathbf{P}^{1,5} &= \begin{bmatrix}
0.9&0.1\\
0.1&0.9
\end{bmatrix},&\mathbf{P}^{2,5} &= \begin{bmatrix}
0.9&0.1\\
0.1&0.9
\end{bmatrix},&\mathbf{P}^{3,5} &= \begin{bmatrix}
0.9&0.1\\
0.1&0.9
\end{bmatrix},&\mathbf{P}^{4,5} &= \begin{bmatrix}
0.9&0.1\\
0.1&0.9
\end{bmatrix},&\mathbf{P}^{5,5} &= \begin{bmatrix}
0.9&0.1\\
0.55&0.45
\end{bmatrix},
\end{align*}

In [18]:
myopicPolicy = genMyopicPolicy(A, r, λ, genS);

In [19]:
heuristic2 = genHeuristic2(r, A, genS)(λ, p);

In [20]:
optimalPolicy, _, gOpt = policyIterationAr(heuristic2, lenS, genS, A, r, p);

In [21]:
_, gmyopic = evalPolicyAr(myopicPolicy, genS, r, p);
_, gh2 = evalPolicyAr(heuristic2, genS, r, p);

In [22]:
gOpt, gmyopic, gh2

(11.200986958462382, 11.361485755956327, 11.209923376395233)